In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         pass
#         print(os.path.join(dirname, filename))
#
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
from pathlib import Path
import shutil
from bs4 import BeautifulSoup
import os

In [3]:
def normalized_coordinates(filename, width, height, xmin, ymin, xmax, ymax):
    """Take in image coordinates (unnormalized) as input, return normalized values 
    """
    
    xmin, xmax = xmin / width, xmax / width
    ymin, ymax = ymin / height, ymax/ height

    width = xmax-xmin
    height = ymax-ymin
    x_center = xmin + (width / 2)
    y_center = ymin + (height / 2)

    return x_center, y_center, width, height

def write_label(filename, x_center, y_center, width, height):
    """Save image's coordinates in text file named "filename"
    """
    with open(filename, mode='w') as outf:
        outf.write(f"{0} {x_center} {y_center} {width} {height}\n")
        
def parse_xml_tags(data):
    """Parse xml label file, return image file name, and its coordinates as a dictionary
    """
    tags = ['filename', 'width', 'height', 'xmin', 'ymin', 'xmax', 'ymax']
    Bs_data = BeautifulSoup(data, "lxml")
    d = dict()

    for t in tags:
        text = Bs_data.find(t).text
        if all(c.isdigit() for c in text):
            d[t] = int(text)
        else:
            d[t] = text
    return d

def build_data(dir_folder, ann_file_list, img_dir):
    """Write xml labels to text file with specifications format, save at 'labels' folder.
        Move image to 'images' folder
    """
    images_folder = f"{dir_folder}/images"
    labels_folder = f"{dir_folder}/labels"
    
    os.makedirs(images_folder, exist_ok = True)
    os.makedirs(labels_folder, exist_ok = True)


    for ann_file in ann_file_list:
        with open(ann_file, 'r') as f:
            label = parse_xml_tags(f.read())
         
        img_file_name = label['filename']
        x_center, y_center, width, height = normalized_coordinates(**label)
         
        # save at 'labels' folder
        write_label(f"{labels_folder}/{img_file_name.split('.')[0]}.txt", x_center, y_center, width, height)
         
         # Move image to 'images' folder
        shutil.copy(f"{img_dir}/{img_file_name}", f"{images_folder}/{img_file_name}")

In [4]:
import glob

dir_folder = "plate_datasets"

ann_list = glob.glob('images/*.xml')
build_data("ultralytics/cfg/datasets/" + dir_folder, ann_list, "images")

In [5]:
import yaml

data = {
    "path": dir_folder,
    "train": "images",
    "val": "images",
    "names": {0: "car_lisence_plate"}
    }

with open('dataset.yaml', 'w') as outfile:
    yaml.dump(data, outfile, default_flow_style=False)

In [6]:
assert len(os.listdir("./ultralytics/cfg/datasets/plate_datasets/labels")) == len(os.listdir("ultralytics/cfg/datasets/plate_datasets/images"))

In [1]:
from ultralytics import YOLO

ImportError: DLL load failed while importing _C: The specified module could not be found.

In [ ]:
model = YOLO("yolov8n.pt" )

d:\Vegeta\Projects\DL projects\Number plate detection\env\Lib\site-packages\ultralytics\nn\tasks.py:781: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(file

In [ ]:
results = model.train(data='dataset.yaml', epochs=30, batch=32, imgsz=640,  project='runs', name='exp',  exist_ok=True, device='cuda')

New https://pypi.org/project/ultralytics/8.2.77 available  Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.55  Python-3.11.9 torch-2.4.0+cu124 CUDA:0 (NVIDIA GeForce GTX 1050, 4096MiB)
engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=dataset.yaml, epochs=30, time=None, patience=100, batch=32, imgsz=640, save=True, save_period=-1, cache=False, device=cuda, workers=8, project=runs, name=exp, exist_ok=True, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=F

d:\Vegeta\Projects\DL projects\Number plate detection\env\Lib\site-packages\ultralytics\nn\tasks.py:781: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(file

AMP: checks skipped . Unable to load YOLOv8n due to possible Ultralytics package modifications. Setting 'amp=True'. If you experience zero-mAP or NaN losses you can disable AMP with amp=False.


d:\Vegeta\Projects\DL projects\Number plate detection\env\Lib\site-packages\ultralytics\engine\trainer.py:267: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(enabled=self.amp)
train: Scanning D:\Vegeta\Projects\DL projects\Number plate detection\ultralytics\cfg\datasets\plate_datasets\labels.cache... 225 images, 0 backgrounds, 0 corrupt: 100%|██████████| 225/225 [00:00<?, ?it/s]


In [ ]:
results= model("TEST\TEST.jpeg")
len(results)


image 1/1 d:\Vegeta\Projects\DL projects\Number plate detection\TEST\TEST.jpeg: 544x640 1 car_lisence_plate, 295.6ms
Speed: 13.8ms preprocess, 295.6ms inference, 1.2ms postprocess per image at shape (1, 3, 544, 640)


1

In [ ]:
results[0].show()

In [1]:
from ultralytics import YOLO
best_model = YOLO("best1.pt")

d:\Vegeta\Projects\DL projects\Number plate detection\env\Lib\site-packages\ultralytics\nn\tasks.py:781: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(file

In [2]:
results= best_model("TEST\TEST.jpeg")
results[0].show()


image 1/1 d:\Vegeta\Projects\DL projects\Number plate detection\TEST\TEST.jpeg: 544x640 1 car_lisence_plate, 4054.9ms
Speed: 176.2ms preprocess, 4054.9ms inference, 145.6ms postprocess per image at shape (1, 3, 544, 640)


In [5]:
results= best_model("TEST\TEST.mp4", stream=True, conf=0.02, iou=0.7)

In [6]:
for i,result in enumerate(results):
    if i == 10:
        break
    if len(result) > 0:
        result.show()


video 1/1 (frame 1/10075) d:\Vegeta\Projects\DL projects\Number plate detection\TEST\TEST.mp4: 384x640 1 car_lisence_plate, 1520.1ms
video 1/1 (frame 2/10075) d:\Vegeta\Projects\DL projects\Number plate detection\TEST\TEST.mp4: 384x640 (no detections), 1517.0ms
video 1/1 (frame 3/10075) d:\Vegeta\Projects\DL projects\Number plate detection\TEST\TEST.mp4: 384x640 (no detections), 1672.3ms
video 1/1 (frame 4/10075) d:\Vegeta\Projects\DL projects\Number plate detection\TEST\TEST.mp4: 384x640 (no detections), 1488.5ms
video 1/1 (frame 5/10075) d:\Vegeta\Projects\DL projects\Number plate detection\TEST\TEST.mp4: 384x640 (no detections), 1214.2ms
video 1/1 (frame 6/10075) d:\Vegeta\Projects\DL projects\Number plate detection\TEST\TEST.mp4: 384x640 (no detections), 1649.5ms
video 1/1 (frame 7/10075) d:\Vegeta\Projects\DL projects\Number plate detection\TEST\TEST.mp4: 384x640 (no detections), 1773.0ms
video 1/1 (frame 8/10075) d:\Vegeta\Projects\DL projects\Number plate detection\TEST\TEST.mp